In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 

import pandas as pd 

In [52]:
browser = webdriver.Firefox() 
url = 'https://www.bbc.com/news'
browser.get(url)
print(f"Successfully visited: {browser.title}")


Successfully visited: BBC News - Breaking news, video and the latest top stories from the U.S. and around the world


In [73]:
browser.quit()

In [ ]:
# Some other approach 

# try:
#     # Wait until the main content is loaded
#     WebDriverWait(browser, 10).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, 'div.gs-c-promo-body'))
#     )
    
#     # Find all article elements
#     articles = browser.find_elements(By.CSS_SELECTOR, 'div.gs-c-promo-body')
    
#     # Extract and print the headline and summary for each article
#     for article in articles:
#         try:
#             headline = article.find_element(By.CSS_SELECTOR, 'h3').text
#             summary = article.find_element(By.CSS_SELECTOR, 'p').text
#             print(f"Headline: {headline}")
#             print(f"Summary: {summary}\n")
#         except Exception as e:
#             print(f"Error extracting article info: {e}")

Removing Cookies 


In [53]:
browser.switch_to.frame(browser.find_element(By.CSS_SELECTOR, "iframe[id^='sp_message_iframe']"))

button1 = browser.find_element(By.CSS_SELECTOR, "button[title='I agree']")
button1.click()
browser.switch_to.default_content()


Changes Region-Section 


In [54]:
button2 = browser.find_element(By.LINK_TEXT , 'Europe')
button2.click()


In [13]:

def scrape_via_next_button(driver, num_pages):
    wait = WebDriverWait(driver, 10)
    data = []
    regions = ['US & Canada', 'UK', 'Africa', 'Asia', 'Australia', 'Europe', 'Latin America', 'Middle East']
    for region in regions :
        print (f'Currently proccessing {region}')
        try :
            region_btn = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, region)))
            region_btn.click()
            time.sleep(3)
        except Exception as e:
            print(f"Could not click region '{region}'. Skipping... Error: {e}")
            continue

        for i in range(1, num_pages + 1):
            print(f"--- Processing Page {i} ---")
            # 1. SCRAPE THE CURRENT PAGE
            # (Your scraping logic goes here)
            links = browser.find_elements(By.TAG_NAME, 'a')
            for link in links:
                try:
                    # 1. Try to find a headline INSIDE this link
                    # (This will fail for menu buttons(headers,footers...), but work for article cards (trageted news "cards")
                    hd = link.find_element(By.CSS_SELECTOR, "[data-testid='card-headline']").text
                    summ= link.find_element(By.CSS_SELECTOR,"[data-testid='card-description']").text
                    url = link.get_attribute('href')
                    data.append  ({
                        'Headlines' : hd ,
                        'Summary' : summ,
                        'URL' : url})
    
                    # print(f"Headline: {hd}")
                    # print(f"Summary: {summ}")
                    # print(f"URL:    {url}")
                    # print("-" * 20)
                except:
                    # If the link doesn't have a headline inside it, just skip it!
                    continue    
            # 2. CLICK NEXT (Don't click on the very last page!)
            if i < num_pages:
                try:
                    # Find the 'Next' button (replace with actual ID/Class/Text)
                    next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,f"button[aria-label='Go to page {i+1}']")))
                    # Scroll to it if necessary
                    driver.execute_script("arguments[0].scrollIntoView();", next_btn)
                    next_btn.click()
                    # Wait for old content to disappear (optional but recommended)
                    time.sleep(2) 
                except Exception as e:
                    print(f"Could not find next button: {e}")
                    break
        
    df = pd.DataFrame(data) 
    return df
    



In [18]:
df = scrape_via_next_button(browser, 12)

Currently proccessing US & Canada
--- Processing Page 1 ---
--- Processing Page 2 ---
--- Processing Page 3 ---
--- Processing Page 4 ---
--- Processing Page 5 ---
--- Processing Page 6 ---
--- Processing Page 7 ---
--- Processing Page 8 ---
--- Processing Page 9 ---
--- Processing Page 10 ---
--- Processing Page 11 ---
--- Processing Page 12 ---
Currently proccessing UK
--- Processing Page 1 ---
--- Processing Page 2 ---
--- Processing Page 3 ---
--- Processing Page 4 ---
--- Processing Page 5 ---
--- Processing Page 6 ---
--- Processing Page 7 ---
--- Processing Page 8 ---
--- Processing Page 9 ---
--- Processing Page 10 ---
--- Processing Page 11 ---
--- Processing Page 12 ---
Currently proccessing Africa
--- Processing Page 1 ---
--- Processing Page 2 ---
--- Processing Page 3 ---
--- Processing Page 4 ---
--- Processing Page 5 ---
--- Processing Page 6 ---
--- Processing Page 7 ---
--- Processing Page 8 ---
--- Processing Page 9 ---
--- Processing Page 10 ---
--- Processing Page 1

In [19]:
df 

,Headlines,Summary,URL
0,Why haven't more Americans faced charges in th...,"Sarah Smith explains why so far, no Americans,...",https://www.bbc.com/news/videos/ce8w279kz6xo
1,Two sisters among those killed in Lake Tahoe a...,More is being learned about the victims even a...,https://www.bbc.com/news/articles/c4g02nxx2plo
2,USA superstar Liu wins women's skating Olympic...,American superstar Alysa Liu adds Olympic figu...,https://www.bbc.com/sport/articles/c33jz026mg3o
3,'Who's next?' - American lawmakers call for 'j...,US lawmakers urge their government to follow t...,https://www.bbc.com/news/articles/c86yj2vjp5go
4,US clinch ice hockey gold with overtime winner,Megan Keller scores in overtime to clinch Wint...,https://www.bbc.com/sport/ice-hockey/articles/...
...,...,...,...
2727,Watch: BBC in Tehran for first time since prot...,"Lyse Doucet reports from Iran, where she says ...",https://www.bbc.com/news/videos/c5yr82796j4o
2728,Watch: Inside Gaza hospital struggling to prov...,More aid has been allowed into Gaza since the ...,https://www.bbc.com/news/videos/c62wpd7wj3ro
2729,Inside Syrian camp holding wives and children ...,"Kurdish-run prisons hold about 8,000 suspected...",https://www.bbc.com/news/videos/c3ve0xd451go
2730,BBC visits UN compound Israel is demolishing i...,John Sudworth says the sounds of heavy machine...,https://www.bbc.com/news/videos/cj0n127y9eqo


In [20]:
print(df.duplicated())

0       False
1       False
2       False
3       False
4       False
        ...  
2727     True
2728     True
2729     True
2730     True
2731    False
Length: 2732, dtype: bool


In [ ]:
## Remove duplicates based on the 'Headlines' column and reset the index
df_clean = df.drop_duplicates(subset=['Headlines']).reset_index(drop=True)


In [36]:
df_clean

,Headlines,Summary,URL
0,Why haven't more Americans faced charges in th...,"Sarah Smith explains why so far, no Americans,...",https://www.bbc.com/news/videos/ce8w279kz6xo
1,Two sisters among those killed in Lake Tahoe a...,More is being learned about the victims even a...,https://www.bbc.com/news/articles/c4g02nxx2plo
2,USA superstar Liu wins women's skating Olympic...,American superstar Alysa Liu adds Olympic figu...,https://www.bbc.com/sport/articles/c33jz026mg3o
3,'Who's next?' - American lawmakers call for 'j...,US lawmakers urge their government to follow t...,https://www.bbc.com/news/articles/c86yj2vjp5go
4,US clinch ice hockey gold with overtime winner,Megan Keller scores in overtime to clinch Wint...,https://www.bbc.com/sport/ice-hockey/articles/...
...,...,...,...
872,Iran leader says protesters are vandals trying...,The largest demonstrations in years have left ...,https://www.bbc.com/news/articles/c4g49djqqjgo
873,Protesters take to the streets of Tehran on Fr...,"The protests, now in their 13th night, erupted...",https://www.bbc.com/news/videos/cj6w82wg416o
874,Why are there huge protests going on in Iran?,Anti-government protests in Iran have continue...,https://www.bbc.com/news/videos/cje1v852evlo
875,Trump's warning looms over Iran protests response,Iranian authorities appear to be unusually res...,https://www.bbc.com/news/articles/c23r4yeyxl9o


In [37]:
df_clean.to_csv('BBC_news.csv', index=False,header=True)

In [51]:
browser.quit()

Scraping the content of an article based on links 

In [61]:
# collect_text =browser.find_elements(By.CSS_SELECTOR, "div[data-component='text-block']")

# for i in collect_text:
#     data_1= []
#     data_1.append(i.text)

# data_1


["But the affair has highlighted what critics argue is the government's attempt to pressurise the broadcaster at the expense of output quality, dubbing Rai TV as TeleMeloni."]

In [ ]:
# Collect web elements that match the CSS selector for text blocks


# This is different than what we did with hd, and herf. url and hd had only one element inside the link, 
# but here we have multiple text blocks that we need to loop through and read each one's text. 
# The text can be scrapped without closing the cookies pop-up, so a button for that was not used here.
def scrape_article_text(url):
    browser = webdriver.Firefox() 
    browser.get(url)

    try:
       headline= WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h1"))).text
    except Exception:
        headline = "No Headline"
    try:
        publish_date = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, "time"))).get_attribute("datetime").text
    except Exception:
        publish_date = "No Date"

    # Some links are /news/videoes and they don't have an author, so I will do a function to get the author 
    authors = ""
    try:
        # The search itself MUST be inside the try block
        raw_authors = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='byline-contributors']"))).text
        # Split and clean up the names
        author_list = raw_authors.split(',')
        for a in author_list:
            authors += a.strip() + "\n"
    except Exception:
        authors = "No Author Listed"

    text_blocks = browser.find_elements(By.CSS_SELECTOR, "div[data-component='text-block']")
    full_article = ""
    for text in text_blocks:
        try : 
            full_article += text.text
        except:
            continue
    the_article = {
        'Headline' : headline,
        'Publish Date' : publish_date,
        'Authors' : authors,
        'Article' : full_article,

    }
    browser.quit()
    return the_article
 

In [ ]:
def scrape_article_text(browser, url):

    browser.get(url)

    try:
        headline = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h1"))).text
    except Exception:
        headline = "No Headline"
        
    try:
        publish_date = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, "time"))).get_attribute("datetime")
    except Exception:
        publish_date = "No Date"

    authors = ""
    try:
        raw_authors = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='byline-contributors']"))).text
        author_list = raw_authors.split(',')
        for a in author_list:
            authors += a.strip() + "\n"
    except Exception:
        authors = "No Author Listed"
    try:
        text_blocks = WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-component='text-block']")))
        full_article = ""
        for text in text_blocks:
            full_article += text.text + "\n" 
    except Exception:
        full_article = "Could not extract text."
    the_article = {
        'Headline' : headline,
        'Publish Date' : publish_date,
        'Authors' : authors,
        'Article' : full_article
    }    

    return the_article

/news/video and sport articles takes the longest time to load and some of their content is not extracted. This happens for 2 reasons, 1. the method we use to exctract the information, 2. the structure of html. 
As an example, sports articles palce the author name if a different spot, compared to the news, 1and /news/video do not have an author name at all. 
Using a different hook to exctract the author name could be a solution.

In [119]:
browser = webdriver.Firefox()
all_detailed_articles = []
for i in df_clean['URL'][:5] :
    print(f"Scraping: {i}") # Helpful to print this so you know it hasn't frozen!
    more= scrape_article_text(browser, i)
    all_detailed_articles.append(more)
browser.quit()
# detailed_df = pd.DataFrame(all_detailed_articles)
# detailed_df.to_csv('BBC_detailed_articles.csv', index=False, header=True)




    


    

Scraping: https://www.bbc.com/news/videos/ce8w279kz6xo
Scraping: https://www.bbc.com/news/articles/c4g02nxx2plo
Scraping: https://www.bbc.com/sport/articles/c33jz026mg3o
Scraping: https://www.bbc.com/news/articles/c86yj2vjp5go
Scraping: https://www.bbc.com/sport/ice-hockey/articles/c07j84jpn55o


In [120]:
all_detailed_articles

[{'Headline': "Why haven't more Americans faced charges in the Epstein case?",
  'Publish Date': '2026-02-19T21:38:06.004Z',
  'Authors': 'No Author Listed',
  'Article': 'Could not extract text.'},
 {'Headline': 'Two sisters among those killed in Lake Tahoe avalanche',
  'Publish Date': '2026-02-20T01:15:26.331Z',
  'Authors': 'Nardine Saad\nLos Angeles\nCalifornia\n',
  'Article': 'The skiers who died in an avalanche in California\'s Lake Tahoe backcountry are said to have been part of a tight-knit group who often went on ski excursions together.\nEight people died after an avalanche struck near Castle Peak in the Sierra Nevada mountains on Tuesday, and rescue teams are searching for a ninth who is presumed dead.\nFamilies named six of the victims on Thursday, saying "they were all mothers, wives and friends, all of whom connected through the love of the outdoors". Six members of the 15-person ski group were rescued early this week.\nThe "football-field sized" avalanche - the deadlie